In [ ]:
import networkx as nx
import random
import math
from itertools import combinations
import pulp




def modularni_produkt(G, H):
    """
    Naredi graf GH, kjer so vozlišča kartezični produkt vozlišč grafa G in H,
    povezave pa so narejene v skladu s tremi pravili v definiciji modularnega
    produkta grafov.

    Parametri:
        G (nx.Graph): Vhodni graf G.
        H (nx.Graph): Vhodni graf H.

    Funkcije vrne:
        nx.Graph: Končni graf GH.
    """

    GH = nx.Graph()
    
    # Vozlišča grafa GH so kartezični produkt V(G) x V(H)
    GH_nodes = [(g, h) for g in G.nodes for h in H.nodes]
    GH.add_nodes_from(GH_nodes)
    
    for g1, h1 in GH_nodes:
        for g2, h2 in GH_nodes:            
            # Tri pravila za preverjanje modularnega produkta
            if (g1 == g2 and (h1, h2) in H.edges) or ((g1, g2) in G.edges and h1 == h2):
                GH.add_edge((g1, h1), (g2, h2))
            elif (g1, g2) in G.edges and (h1, h2) in H.edges:
                GH.add_edge((g1, h1), (g2, h2))
            elif (g1 != g2 and h1 != h2) and (g1, g2) not in G.edges and (h1, h2) not in H.edges:
                GH.add_edge((g1, h1), (g2, h2))
    
    return GH






def dominacijsko_stevilo(G: nx.Graph) -> int:
    """
    Najde dominacijsko število grafa s pomočjo linearnega programiranja.

    Parametri:
        G (nx.Graph): Vhodni graf G.

    Funkcija vrne dominacijsko število grafa.
    """
    # Naredimo linearni program
    lp = pulp.LpProblem("DominacijskoStevilo", pulp.LpMinimize)

    # Za vsako vozlišče definiramo odločitveno binarno spremenljivko x (bodisi enaka 1 ali 0)
    x = {vozlisce: pulp.LpVariable(f"x_{vozlisce}", cat="Binary") for vozlisce in G.nodes}

    # Hočemo minimizirati vsoto odločitvenih spremenljivk
    lp += pulp.lpSum(x.values())

    # Pogoji: Vsako vozlišče mora biti dominirano
    for vozlisce in G.nodes:
        # Vozlišče je dominirano, če je element dominacijske množice ali če je
        # njegov sosed element dominacijske množice
        lp += pulp.lpSum(x[sosed] for sosed in G.neighbors(vozlisce)) + x[vozlisce] >= 1

    # Rešimo linearni program
    lp.solve()

    # Programa vrne dominacijsko število (minimalna moč dominacijske množice grafa)
    return int(pulp.value(lp.objective))







from sage.all import *

def is_sb(G: Graph) -> bool:
    # Premer mora biti 2
    if G.diameter() != 2:
        return False

    # Matrika sosednosti
    adj = G.adjacency_matrix()

    # Iteriramo čez vse pare sosednjih vozlišč
    for i in range(adj.nrows()):  
        for j in range(i + 1, adj.nrows()):  # Samo zgornji trikotnik matrike
            if adj[i, j] == 1:  # Če sta i in j soseda
                # Preverimo, če nimata skupnih sosedov
                common_neighbour = any(adj[i, k] and adj[j, k] for k in range(adj.nrows()))
                if common_neighbour:
                    continue
                
                # Poiščemo vozlišče, ki ni povezano z i in j
                nonadj_vertex_exists = any(
                    not adj[i, k] and not adj[j, k] 
                    for k in range(adj.nrows()) 
                    if k != i and k != j
                )
                if nonadj_vertex_exists:
                    return True

    # Če noben par ni ustrezal, graf ni tipa (SB)
    return False






def generate_random_sb(n):
    # TODO: does changing probability make much of a difference? probably not
    g = graphs.RandomGNP(n, 0.5)
    h1, h2, _ = g.edges()[0] # we take a random edge
    # and mark those two vertices as h1 and h2
    a1 = g.neighbors(h1)
    a2 = g.neighbors(h2)
    a1.remove(h2)
    a2.remove(h1)

    a_star = []
    for v in g.vertices():
        if v not in a1 and v not in a2 and v != h1 and v != h2:
            a_star.append(v)

    vertices_to_remove = []
    for n1 in a1:
        for n2 in a2:
            if n1 == n2:
                # delete edge
                # does it matter, from which h we delete? i dont think so
                g.delete_edge(h1, n1)
                # this way we got rid of commmon neighbours
                vertices_to_remove.append(n1)

    # we remove the affected vertices from our a1 set
    for v in vertices_to_remove:
        a1.remove(v)

    if len(a1) == 0:
        # add vertex into a1
        # also add edge from h1 to newly added vertex
        u = g.add_vertex()
        a1.append(u)
        g.add_edge(h1, u)

    if len(a2) == 0:
        u = g.add_vertex()
        a2.append(u)
        g.add_edge(h2, u)

    if len(a_star) == 0:
        u = g.add_vertex()
        a_star.append(u)
        v_a1 = a1[0]
        v_a2 = a2[0]
        g.add_edge(u, v_a1)
        g.add_edge(u, v_a2)

    # each vertex from a1/a2 needs to be connected to each vertex in a_star
    # otherwise graph cannot be type SB! diameter won't be 2
    for u in a1:
        for v in a_star:
            g.add_edge(u, v)

    # same for other set
    for u in a2:
        for v in a_star:
            g.add_edge(u, v)

    # should be 2 if we didn't go wrong somewhere
    diam = g.diameter()

    if diam == 2 and is_sb(g):
        return g
    else:
        # debugging
        print("Something went wrong!")
        print("Diameter: ", diam)
        print("h1: ", h1, ", h2: ", h2)
        print("A1: ", a1)
        print("A2: ", a2)
        print("A*: ", a_star)
        print(g.vertices())
        print(g.edges())






import random

# Uvoziti moramo še potrebne funkcije iz ostalih dokumentov

def random_modify_sb_graph(G):
    #Naključno spremeni graf tipa (SB) in ustvari nov graf tipa (SB).
    
    original_graph = G.copy()  # Shrani originalni graf, če se zacikla
    
    # Preverimo, ali je graf že tipa (SB)
    if not is_sb(G):
        print("Graf ni tipa (SB). Ne bomo izvajali sprememb.")
        return None
    
    # Particija vozlišč
    vertices = list(G.vertices())
    h1, h2 = None, None
    while h1 is None or h2 is None or not G.has_edge(h1, h2) or set(G.neighbors(h1)) & set(G.neighbors(h2)):
        h1, h2 = random.sample(vertices, 2)

    A1 = [v for v in G.neighbors(h1) if v != h2]
    A2 = [v for v in G.neighbors(h2) if v != h1]
    Astar = [v for v in vertices if v not in {h1, h2} and v not in A1 and v not in A2]

    
    i = 0
    while i < 1000:  # Omejitev poskusov
        i += 1
        for _ in range(random.randint(1, 5)):  # Naključno število sprememb, lahko bi jih bilo več
            action = random.choice(["add_edge", "remove_edge", "add_vertex", "remove_vertex"])

            if action == "add_edge":
                u, v = random.sample(vertices, 2)
                if u != v and not G.has_edge(u, v): # Da ne dela zanke vase
                    G.add_edge(u, v)

            elif action == "remove_edge":
                edges = list(G.edges())
                if edges:
                    edge = random.choice(edges)
                    u, v = edge[0], edge[1] # Ne vzamemo tretje komponente None
                    if u != v:
                        G.delete_edge(u, v)

            elif action == "add_vertex":
                new_vertex = G.order() # Indeks novega vozlišča je število vozlišč v prejšnjem grafu (ker štejemo začenši z 0)
                G.add_vertex(new_vertex)
                vertices.append(new_vertex)
                for vertex in random.sample([h1, h2] + A1 + A2 + Astar, random.randint(1, len([h1, h2] + A1 + A2 + Astar))): # Povežemo lahko z vsemi razen s sabo
                    if new_vertex != vertex:
                        G.add_edge(new_vertex, vertex)

            elif action == "remove_vertex":
                removable_vertices = [v for v in vertices if v not in {h1, h2}] # Ne odstranimo najpomembnejših vozlišč, ker je potem problem lahko precej daljši za velike n
                if removable_vertices:
                    to_remove = random.choice(removable_vertices)
                    if to_remove in G.vertices():
                        G.delete_vertex(to_remove)
                        vertices.remove(to_remove)

        # Preverimo, ali je graf še vedno tipa (SB)
        if is_sb(G):
            return G
        print(f"Poskus {i}: Graf ni tipa (SB), poskušamo znova...")

    print("Dosežena omejitev poskusov. Poskušamo ponovno z originalnim grafom.")
    return random_modify_sb_graph(original_graph)